In [ ]:
cd 'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\Biofilm growth code\GSWL code\Biochemical-databases'

# TECRDB scraping 

In [ ]:
# import libraries
from bs4 import BeautifulSoup
from time import sleep
import requests
import datetime
import pandas
import numpy
import glob
import math
import os
import re

# defining the website
root_url = "https://randr.nist.gov/enzyme/DataDetails.aspx?ID="
end_url = "&finalterm=&data=enzyme"
#===========================================================================================================================

# identify the table and rows of pertinent data
bs = BeautifulSoup(open('Enzyme Thermodynamic Database.html'), 'lxml')
table = bs.find("table", attrs = {'id': 'MainBody_gvSearch'})
body = table.find_all("tr")

# defining the boundaries of the dataframe section
index_range = 12000
index_count = 0
loop_count = 0 
output_loop = 1
total_entries = math.floor(1*len(body))

# loop through the enzyme id values 
scraped_csv_folder_path = './individual scraping/'
name_iteration = 0
enzyme_iteration = 0
for row in range(1, total_entries):   
    
    # parsing the reaction names and strings   
    enzyme_name = body[row].find('span', attrs = {'id': 'MainBody_gvSearch_lblEnzyme_%s' %(name_iteration)}).text
    reaction = body[row].find('span', attrs = {'id': 'MainBody_gvSearch_lblReaction_%s' %(name_iteration)}).text
    id_value = body[row].find("a").text
    name_iteration += 1
    
    # defining the soup
    total_url = root_url + id_value + end_url
    soup = requests.get(total_url).text
    bs = BeautifulSoup(soup, 'lxml')
    
    # scrape the table and header information
    tables1 = bs.find_all("table", attrs = {"id": "MainBody_extraData"})
    print(id_value, '\t: ', loop_count, '\t, ', len(tables1))
    if len(tables1) != 1:
        continue

    body1 = tables1[0].find_all("tr")
    body_rows1 = body1[1:]
    heads = body1[0]

    headings = ['Enzyme:', 'Reaction:']
    for head in heads.find_all("th"):
        head = (head.text).rstrip("\n")
        headings.append(head)

    total_rows = []
    for row_number in range(len(body_rows1)):
        each_row = [enzyme_name, reaction]
        for row_element in body_rows1[row_number].find_all("td"):
            row_refined = re.sub("(\xa0)|(\n)|,","",row_element.text)
            each_row.append(row_refined)
        total_rows.append(each_row)
    
    # create a dataframe
    index_list_body = range(index_count+lower_bound-1, len(body_rows1)+index_count+lower_bound-1)
    bs_dataframe_table1 = pandas.DataFrame(data = total_rows, columns = headings, index = index_list_body)
    bs_dataframe_table1.drop(bs_dataframe_table1.columns[len(bs_dataframe_table1.columns)-1], axis=1, inplace=True)
    '''display(bs_dataframe_table1)'''
    
#===========================================================================================================================

    # scrape additional information 
    tables = bs.find_all("table", attrs={"id": "MainBody_DataList1"})
    if len(tables) == 0:
        print('Failed reference ID: ', id_value)
        continue
        
    body2 = tables[0].find_all("tr")
    body_rows2 = body2[1:]

    each_row2 = []
    for row in range(len(body_rows2)):
        for row_element in body_rows2[row].find_all("td"):
            row_refined2 = re.sub("(\xa0)|(\n)|,","",row_element.text)
            each_row2.append(row_refined2)

    information_entries_list = []
    information_values_list = []
    column_count = 0
    for i, element in enumerate(each_row2):
        if i == 0 or i % 2 == 0:
            information_entries_list.append(element)
            column_count += 1
        else:
            information_values_list.append(element)
            column_count += 1
    column_count /= 2

    # create the dataframe and refine the columns
    index_list_reference = range(index_count+lower_bound-1, index_count+1+lower_bound-1)
    bs_dataframe_pretable2 = pandas.DataFrame(data = [information_values_list], columns = information_entries_list, index = index_list_reference)
    bs_dataframe_pretable2.drop(bs_dataframe_pretable2.columns[len(bs_dataframe_pretable2.columns)-2], axis=1, inplace=True)
    bs_dataframe_pretable2.drop(bs_dataframe_pretable2.columns[len(bs_dataframe_pretable2.columns)-1], axis=1, inplace=True)
    
#===========================================================================================================================

    # merge the dataframes of this loop
    this_dataframe = bs_dataframe_table1.join(bs_dataframe_pretable2)
    this_dataframe.index.name = 'index'
    
    # iteratively coalesce the new dataframe into the old dataframe 
    if loop_count == 0:
        old_dataframe = this_dataframe
        old_dataframe.index.name = 'index'
        
    elif loop_count > 0:
        these_columns = []
        for column in this_dataframe:
            these_columns.append(column)
            
        old_columns = []
        for existing_column in old_dataframe:
            old_columns.append(existing_column)
            
        common_columns = list(set(these_columns).intersection(old_columns))
        
        # amalgamate the dataframe with the pre-existing dataframe
        current_dataframe = old_dataframe.merge(this_dataframe, on = common_columns, how = 'outer')
        old_dataframe = current_dataframe  
        
    # amalgamate the dataframe with the pre-existing dataframe
    index_count += len(body_rows1)
    
    time_delay = 0
    sleep(time_delay)
    
    max_referenes_per_csv = 0
    if loop_count == max_referenes_per_csv:
        id_value = re.sub('(/)', '-', id_value)
        output = '{}/{}, {}.csv'.format(scraped_csv_folder_path, datetime.date.today(), id_value, output_loop)
        while os.path.exists(output):
            output_loop += 1
            output = '{}/2021-05-06_{}.csv'.format(scraped_csv_folder_path, datetime.date.today(), id_value, output_loop)
            
        old_dataframe.to_csv(output)
        loop_count = 0 
    else:        
        loop_count += 1 


# create the total list of dataframes
files = glob.glob(os.path.join(scraped_csv_folder_path, '*.csv'))
total_dataframes = []
for file in files:
    dfn = pandas.read_csv(file)
    total_dataframes.append(dfn)
    
# combine the total set of dataframes
combined_df = pandas.DataFrame()
combined_df = pandas.concat(total_dataframes)
display(combined_df)

# refine the dataframe 
combined_df = combined_df.fillna(' ')

middle_dataframe_columns = ['T(K)', 'pH ', 'K<sub>c</sub>\' ', 'δ<sub>r</sub>H\'<sup>o</sup>(kJ.mol<sup>-1</sup>)', 'Km\'']
left_dataframe_columns = ['index', 'Enzyme:', 'Reaction:', 'Reference:', 'Reference ID:'] 
right_dataframe_columns = list(set(combined_df.columns) - set(left_dataframe_columns) - set(middle_dataframe_columns))
defined_columns = left_dataframe_columns + middle_dataframe_columns + right_dataframe_columns  
final_dataframe = combined_df.reindex(columns = defined_columns)

# export the final dataframe of values
'''final_dataframe.to_csv('{}_concatenated scraped NIST enzymes_01.csv'.format(datetime.date.today()))'''

# Data assemblage and organization

In [2]:
#import libraries
from datetime import date 
from time import sleep
import pandas
import numpy
import math
import json
import re

empty_cell = ['nan', 'NaN', 'none', 'not given', 'unknown', '0', '', None, ' ']

# dataframes definition
combined_dataframe = pandas.read_csv("2021-05-06_concatenated scraped NIST enzymes_01.csv")
df = combined_dataframe.astype(str)


# combine columns of the complete scraped excel file 
combined_columns = []
regex_column_searches = ['(?i)(^K)', '(?i)(^K)', '(ë«|Î\´|δ)', '(I<sub>c)']
base_columns = ['K<sub>c</sub>\' ', 'K<sub>c</sub>\' ', 'δ<sub>r</sub>H(cal)/kJ mol<sup>-1</sup>)', 'I<sub>c</sub>(mol dm<sup>-3</sup>)']
print('\nColumns:\n', '='*len('Columns:'))
for this_column in df:
    print(this_column)
    for index, row in df.iterrows():
        for search in range(len(regex_column_searches)):
            
            # combine similar columns
            if re.search(regex_column_searches[search], this_column) and not re.search('(Km\')', this_column):
                if str(df.at[index, this_column]) not in empty_cell: 
                    if str(df.at[index, base_columns[search]]) in empty_cell:
                        df.at[index, base_columns[search]] = str(df.at[index, this_column])

                    if str(df.at[index, this_column]) not in empty_cell:
                        if str(df.at[index, base_columns[search]]) != str(df.at[index, this_column]):
                            df.at[index, base_columns[search]] = str(df.at[index, base_columns[search]]) + ' & ' + str(df.at[index, this_column])

                if this_column !=  base_columns[search]:
                    if this_column not in combined_columns:
                        combined_columns.append(this_column)

        if re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column):
            solute = str(re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column).group(1))
                
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) in empty_cell:
                    df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)'] = str(df.at[index, this_column]) + ' ' + solute

                if str(df.at[index, this_column]) not in empty_cell:
                    if str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) != (str(df.at[index, this_column]) or str(df.at[index, this_column]) + ' ' + solute):
                        df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)'] = str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) + ' & ' + str(df.at[index, this_column]) + ' ' + solute

            if this_column !=  'c(glycerol,mol dm<sup>-3</sup>)':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        if re.search('(?<=m\()(\w+\d?\+?)(?<!,)', this_column):
            solute = str(re.search('(?<=m\()(\w+\d?\+?)(?<!,)', this_column).group(1))
                
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) in empty_cell:
                    df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)'] = str(df.at[index, this_column]) + ' ' + solute
                if str(df.at[index, this_column]) not in empty_cell:
                    if str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) != (str(df.at[index, this_column]) or str(df.at[index, this_column]) + ' ' + solute):
                        df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)'] = str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) + ' & ' + str(df.at[index, this_column]) + ' ' + solute

            if this_column !=  'm(MgCl2,mol.kg<sup>-1</sup>)':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        buffer_columns = ['buffer(mol dm<sup>-3</sup>)', 'buffer and/or salt ', 'media ', 'buffer ']
        if this_column in buffer_columns:
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'Buffer:']) in empty_cell:
                    df.at[index, 'Buffer:'] = str(df.at[index, this_column])

                if str(df.at[index, 'Buffer:']) not in empty_cell:
                    if not re.search(re.escape(str(df.at[index, this_column])), str(df.at[index, 'Buffer:'])):
                        df.at[index, 'Buffer:'] = str(df.at[index, 'Buffer:']) + '  +  ' + str(df.at[index, this_column])

            if this_column !=  'Buffer:':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        solution_columns = ['salt ', 'cosolvent ', 'added solute ', 'protein ', 'added solute ', 'percent(dimethyl sulfoxide) ', 'p(MPa)', 'pMg ']
        if this_column in solution_columns:
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'solvent ']) in empty_cell:
                    if str(this_column) == 'p(MPa)':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' megapascals'  
                    elif str(this_column) == 'pMg ':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' = -log[Mg+2]'   
                    elif str(this_column) == 'percent(dimethyl sulfoxide) ':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' % DMSO'   
                    else:
                        df.at[index, 'solvent '] = str(df.at[index, this_column])

                if str(df.at[index, 'solvent ']) not in empty_cell:
                    if not re.search(re.escape(str(df.at[index, this_column])), str(df.at[index, 'solvent '])):
                        df.at[index, 'solvent '] = str(df.at[index, 'solvent ']) + '  +  ' + str(df.at[index, this_column])

            if this_column !=  'solvent ':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)
            
# rename the base columns
df.rename(columns = {'c(glycerol,mol dm<sup>-3</sup>)':'solutes [mol / dm^3]', 
                     'I<sub>c</sub>(mol dm<sup>-3</sup>)':'Ionic strength [mol / dm^3]', 
                     'T(K)':'T [K]', 
                     'I<sub>m</sub>(mol.kg<sup>-1</sup>)':'Ionic strength [mol / kg]', 
                     'm(MgCl2,mol.kg<sup>-1</sup>)':'solutes [mol / kg]', 
                     'solvent ':'Experimental conditions', 
                     'K<sub>c</sub>\' ':'Keq', 
                     'δ<sub>r</sub>H(cal)/kJ mol<sup>-1</sup>)':'Enthalpy [kJ / mol]',
                     'Km\' ':'Km'},
          inplace = True)
          
    
# delete the combined columns
print('\nDeleted columns:')
for column in combined_columns:
    print(column)
    del df[column]
del df['Km\'']
    
for column in df.columns:
    if re.search('|index|Unnamed', column):
        print(column)
        del df[column]
        
# export the dataframe to csv 
'''df.to_csv("{}_vetted&reorganized_TECR.csv".format(date.today()))'''


# acquire a list of all enzymes
enzyme_list = []
empty_cell = ['nan', 'NaN', 'none', 'not given', '', ' ', None, numpy.nan]
for index, row in df.iterrows():
    if df.at[index, 'Enzyme:'] not in enzyme_list and df.at[index, 'Enzyme:'] not in empty_cell:
        enzyme_list.append(df.at[index, 'Enzyme:'])      
        
enzymes = []
for original_enzyme in enzyme_list:
    enzyme_name = re.search('(\w.*)',original_enzyme)
    enzymes.append(enzyme_name.group())
    
# count down for processing and organizing the data
output_json_name = '{}_TECR_consolidated.json'.format(date.today())

count = 0
'''while not os.path.exists(output_json_name):
    if count % 2:
        mark = '.'
    else:
        mark = ''
    print('The data is being assembled and organized ..{}'.format(mark), end = '\r')
    count += 1
    sleep(1)'''
    
'''necessary_time_to_organize = 460
for time in reversed(range(necessary_time_to_organize)):
    print('The database data is being assembled and organized ... {}'.format(time), end = '\r')
    time -= 1
    sleep(1)'''
    
# the data is processed and organized from the originally scraped data 
data_per_enzyme = {}
total_entries = 3979
count = 1
        
for enzyme in enzymes:
    
    # print the progress statement
    print('The data is being assembled and organized ... {}/425'.format(count), end = '\r')
    
    # lists of the database variables
    keq_values_per_enzyme = []
    km_values_per_enzyme = []
    enthalpy_values_per_enzyme = []
    temperatures_per_enzyme = []
    phs_per_enzyme = []
    
    # lists of identifying whether the reference contains the identified variable 
    references_of_an_enzyme = []
    reaction_of_an_enzyme = []
    km_in_the_reference = []
    enthalpy_in_the_reference = []
    keqs_in_a_reference = []
    for index, row in df.iterrows():
        iteration = 0
        if df.at[index, 'Enzyme:'] == ' %s' %(enzyme):
            reaction_of_an_enzyme.append(df.at[index, 'Reaction:'])
            if df.at[index, 'Reference:'] not in empty_cell:
                references_of_an_enzyme.append(df.at[index, 'Reference:'])
            elif df.at[index, 'Reference:'] in empty_cell:
                references_of_an_enzyme.append('Ibid')
            
            # clean keqs are added to a list
            if df.at[index, 'Keq'] not in empty_cell:
                cleaned_keq = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Keq']))
                keq_values_per_enzyme.append(float(cleaned_keq.group())) 
                keqs_in_a_reference.append('True')
                temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                if df.at[index, 'pH '] not in empty_cell:
                    phs_per_enzyme.append(df.at[index, 'pH '])
                elif df.at[index, 'pH '] in empty_cell:
                    phs_per_enzyme.append('nan')
            
            elif df.at[index, 'Keq'] in empty_cell:
                keqs_in_a_reference.append('False')    
                
            # clean kms are added to a list
            if df.at[index, 'Km'] not in empty_cell:
                cleaned_km = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Km']))
                km_values_per_enzyme.append(float(cleaned_km.group())) 
                if df.at[index, 'Keq'] in empty_cell:
                    temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                    if df.at[index, 'pH '] not in empty_cell:
                        phs_per_enzyme.append(df.at[index, 'pH '])
                    elif df.at[index, 'pH '] in empty_cell:
                        phs_per_enzyme.append('nan')
                    
            elif df.at[index, 'Km'] in empty_cell:
                km_in_the_reference.append('False')
                
            # clean enthalpy values are added to a list
            if df.at[index, 'Enthalpy [kJ / mol]'] not in empty_cell:
                cleaned_enthalpy = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Enthalpy [kJ / mol]']))
                enthalpy_in_the_reference.append('True')
                enthalpy_values_per_enzyme.append(float(cleaned_enthalpy.group())) 
                if df.at[index, 'Keq'] in empty_cell and df.at[index, 'Km'] in empty_cell:
                    temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                    if df.at[index, 'pH '] not in empty_cell:
                        phs_per_enzyme.append(df.at[index, 'pH '])
                    elif df.at[index, 'pH '] in empty_cell:
                        phs_per_enzyme.append('nan')
                    
            elif df.at[index, 'Enthalpy [kJ / mol]'] in empty_cell:
                enthalpy_in_the_reference.append('False')
                
                
            #loop through the unlabeled rows of each enzyme
            while df.at[index + iteration, 'Enzyme:'] in empty_cell:
                if df.at[index, 'Keq'] not in empty_cell:
                    cleaned_keq = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Keq']))
                    keq_values_per_enzyme.append(float(cleaned_keq.group())) 
                    keqs_in_a_reference.append('True')
                    temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                    if df.at[index, 'pH '] not in empty_cell:
                        phs_per_enzyme.append(df.at[index, 'pH '])
                    elif df.at[index, 'pH '] in empty_cell:
                        phs_per_enzyme.append('nan')

                elif df.at[index, 'Keq'] in empty_cell:
                    keqs_in_a_reference.append('False')  
                    
                #clean kms are added to a list
                if df.at[index, 'Km'] not in empty_cell:
                    cleaned_km = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Km']))
                    km_in_a_reference.append('True')
                    km_values_per_enzyme.append(float(cleaned_km.group())) 
                    if df.at[index, 'Keq'] in empty_cell:
                        temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                        if df.at[index, 'pH '] not in empty_cell:
                            phs_per_enzyme.append(df.at[index, 'pH '])
                        elif df.at[index, 'pH '] in empty_cell:
                            phs_per_enzyme.append('nan')

                elif df.at[index, 'Km'] in empty_cell:
                    km_in_the_reference.append('False')
                    
                # clean enthalpy values are added to a list
                if df.at[index, 'Enthalpy [kJ / mol]'] not in empty_cell:
                    cleaned_ethalpy = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Enthalpy [kJ / mol]']))
                    enthalpy_in_the_reference.append('True')
                    enthalpy_values_per_enzyme.append(float(cleaned_enthalpy.group())) 
                    if df.at[index, 'Keq'] in empty_cell and df.at[index, 'Km'] in empty_cell:
                        temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                        if df.at[index, 'pH '] not in empty_cell:
                            phs_per_enzyme.append(df.at[index, 'pH '])
                        elif df.at[index, 'pH '] in empty_cell:
                            phs_per_enzyme.append('nan')

                elif df.at[index, 'Enthalpy [kJ / mol]'] in empty_cell:
                    enthalpy_in_the_reference.append('False')
                    
                #proceed to the next loop
                if iteration + index < total_entries:
                    iteration += 1

    #processing the average and standard deviation Keq values
    if len(keq_values_per_enzyme) != 0:
        average_keq_per_enzyme = sum(keq_values_per_enzyme) / len(keq_values_per_enzyme)
        standard_deviation_keq_per_enzyme = math.sqrt(sum([(x - average_keq_per_enzyme)**2 for x in keq_values_per_enzyme]) / len(keq_values_per_enzyme))
    elif len(keq_values_per_enzyme) == 0:
        average_keq_per_enzyme = 'nan'
        standard_deviation_keq_per_enzyme = 'nan'
        
    #processing the average and standard deviation Km values
    if len(km_values_per_enzyme) != 0:
        average_km_per_enzyme = sum(km_values_per_enzyme) / len(km_values_per_enzyme)
        standard_deviation_km_per_enzyme = math.sqrt(sum([(x - average_km_per_enzyme)**2 for x in km_values_per_enzyme]) / len(km_values_per_enzyme))
    elif len(km_values_per_enzyme) == 0:
        average_km_per_enzyme = 'nan'
        standard_deviation_km_per_enzyme = 'nan' 
        
    #processing the average and standard deviation enthalpy values
    if len(enthalpy_values_per_enzyme) != 0:
        average_enthalpy_per_enzyme = sum(enthalpy_values_per_enzyme) / len(enthalpy_values_per_enzyme)
        standard_deviation_enthalpy_per_enzyme = math.sqrt(sum([(x - average_enthalpy_per_enzyme)**2 for x in enthalpy_values_per_enzyme]) / len(enthalpy_values_per_enzyme))
    elif len(enthalpy_values_per_enzyme) == 0:
        average_enthalpy_per_enzyme = 'nan'
        standard_deviation_enthalpy_per_enzyme = 'nan' 
        
    #store the information into a nested dictionary structure
    data_per_enzyme[enzyme] = {'reaction':reaction_of_an_enzyme,
                               'experimental temperatures':temperatures_per_enzyme,
                               'experimental phs':phs_per_enzyme,
                               'keq reference':references_of_an_enzyme,
                               'Keq':{'keq values in the reference':keqs_in_a_reference,
                                       'keqs':keq_values_per_enzyme, 
                                       'keq quantity':len(keq_values_per_enzyme), 
                                       'keq average':average_keq_per_enzyme, 
                                       'keq standard deviation':standard_deviation_keq_per_enzyme},
                               'Km':{'km values in the reference':km_in_the_reference,
                                    'km values':km_values_per_enzyme,
                                    'km average':average_km_per_enzyme,
                                    'km standard deviation':standard_deviation_km_per_enzyme},
                               'Enthalpy':{'enthalpy values in the reference':enthalpy_in_the_reference,
                                         'enthalpy values':enthalpy_values_per_enzyme,
                                         'enthalpy average':average_enthalpy_per_enzyme,
                                         'enthalpy standard deviation':standard_deviation_enthalpy_per_enzyme}
                              }
    count += 1

#export the database dictionary as a JSON file
with open(output_json_name, 'w') as output:
    json.dump(data_per_enzyme, output, indent = 4)


Columns:
Unnamed: 0
index
Enzyme:
Reaction:
Reference:
Reference ID:
T(K)
pH 
K<sub>c</sub>' 
δ<sub>r</sub>H'<sup>o</sup>(kJ.mol<sup>-1</sup>)
Km'
δ<sub>r</sub>H(kJ.mol<sup>-1</sup>)
K<sub>c</sub> 
m(KCl,mol.kg<sup>-3</sup>)
added solute 
δ<sub>r</sub>H<sup>o</sup>(kJ.mol<sup>-1</sup>)
cosolvent 
δ<sub>r</sub>H?(kJ.mol<sup>-1</sup>)
Km' 
media 
Method:
salt 
c(Mg)tot(mol dm<sup>-3</sup>)
c(Tris,mol dm<sup>-3</sup>)
p(MPa)
c(NaCl,mol dm<sup>-3</sup>)
I<sub>c</sub> 
c(MgCl2,mol dm<sup>-3</sup>)
buffer and/or salt 
Buffer:
K 
c(CaCl2,mol dm<sup>-3</sup>)
buffer(mol dm<sup>-3</sup>)
solvent 
c(MgSO4,mol dm<sup>-3</sup>)
I<sub>c</sub>(kJ.mol<sup>-1</sup>)
δ<sub>r</sub>H(cal)/(n<sup>-1</sup>
kJ.mol<sup>-1</sup>)
c(Mg2+,mol dm<sup>-3</sup>)
c(salt,mol dm<sup>-3</sup>)
pMg 
δ<sub>r</sub>H(cal)/,kJ.mol<sup>-1</sup>)
EC Value:
δ<sub>r</sub>Hcal,kJ.mol<sup>-1</sup>)
K' 
c(phosphate,mol dm<sup>-3</sup>)
percent(dimethyl sulfoxide) 
c(total substrate,mol dm<sup>-3</sup>)
δ<sub>r</sub>H(cal) 
m(MgC

# ModelSEED reconfiguration

In [3]:
# import the requisite libraries
from ftfy import fix_encoding
from datetime import date
import pandas
from numpy import nan
import json
import re
import io


# import and define reused information
tecrdb_compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))
tecrdb_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))
nist = json.load(open('2021-05-06_NIST consolidated.json'))
empty_cells = ['null', ' null', nan, 'None', None, ""]

# create a condensed dictionary of all ModelSEED compound names and aliases
modelseed = json.load(open('compounds.json'))
modelseed_compounds = {}
all_names = []
for compound2 in modelseed:
    if compound2['is_obsolete'] == 0:
        try:
            for entry in compound2['aliases']:
                if re.search('Name:', entry):
                    alternative_names = re.sub('Name: ', '', entry).split('; ')
                    alternative_names.append(compound2['abbreviation'])
                    main_name = alternative_names[0].lower()
                if re.search('KEGG:', entry):
                    kegg_id = re.sub('KEGG: ', '', entry).split('; ')

        except:
            main_name = compound2['name'].lower()
            alternative_names = compound2['abbreviation']
            
        modelseed_compounds[main_name] = {'ModelSEED ID': compound2['id'],
                                          'KEGG ID': kegg_id,
                                          'alternative names': alternative_names}

# export the dictionary of modelseed compounds with the respective information
'''with open('{}_modelseed compounds & alternative names_01.json'.format(date.today()), 'w') as output:
    json.dump(modelseed_compounds, output, indent = 4)'''
    
    
# parse the modelseed database list of compounds
all_names = []
for compound in modelseed_compounds:
    if type(modelseed_compounds[compound]['alternative names']) is list:
        all_names.append([compound.lower()] + modelseed_compounds[compound]['alternative names'])
    elif type(modelseed_compounds[compound]['alternative names']) is str:
        all_names.append([compound.lower()] + list(modelseed_compounds[compound]['alternative names']))
    else:
        print('ERROR: The database entry for {} is an unpredicted structure.')
        
total_modelseed_names = []
for name_list in all_names:
    total_modelseed_names += name_list 
total_modelseed_names_string = ',\t'.join(list(set(total_modelseed_names)))   


# export the string of identified ModelSEED compound names
'''with open('2021-06-27_total_modelseed_names_string_01.txt', 'w') as output:
    output.write(modelseed_abbreviations_string)
    output.close()'''
modelseed_abbreviations = modelseed_compounds.keys()
modelseed_abbreviations_string = ',\t'.join(modelseed_abbreviations)
'''with open('2021-06-27_modelseed abbreviation string_01.txt', 'w') as output:
    output.write(modelseed_abbreviations_string)
    output.close()'''


# determine the set of described and undescribed compounds of the TECRDB in the ModelSEED database 
described_compounds = {}
undescribed_compounds = []
unknown_compounds = []
repeated_compounds = set()
for compound in tecrdb_compounds:    
    compound = re.sub('(\(\+\))', '\(\+\)', compound)
    compound = compound.lower()
    if compound not in (described_compounds.keys() or undescribed_compounds or unknown_compounds):
        if re.search('((^|,\t){},\t)'.format(compound), total_modelseed_names_string, re.IGNORECASE):
            if re.search('((^|,\t){},\t)'.format(compound), modelseed_abbreviations_string, re.IGNORECASE):
                compound = fix_encoding(compound)
                print('identified abbreviation: ', compound)
                #re.sub(',\s(\w)\w+', '\U\1', ', {}, '.format(compound))
                compound = re.sub('(\\\)', '', compound)
                described_compounds[compound] = modelseed_compounds[compound]
            else:
                for name_list in all_names:
                    name_list_string = ',\t'.join(name_list)
                    if re.search('((^|,\t){},\t)'.format(compound), name_list_string, re.IGNORECASE):
                        root_compound = name_list[0]
                        root_compound = fix_encoding(root_compound)
                        print('identified alias: ', compound)
                        #print(name_string)
                        described_compounds[compound] = modelseed_compounds[root_compound]
    
                if compound not in (described_compounds.keys() or unknown_compounds):
                    unknown_compounds.append(compound)
                    print('ERROR 1: {} is structurally predicted.'.format(compound))
                    
        else:
            if compound not in undescribed_compounds:
                compound = fix_encoding(compound)
                undescribed_compounds.append(compound)
                print('--> undescribed compound in ModelSEED, {} '.format(compound))
            else:
                if compound not in unknown_compounds:
                    unknown_compounds.append(compound)
                    print('ERROR 2: {} is structurally predicted.'.format(compound))
                    
    else:
        repeated_compounds.add(compound)
                
            
# view the parsed data
print('\n\n')
compounds_list = []
for compound in tecrdb_compounds.keys():
    compound = fix_encoding(compound)
    compounds_list.append(compound.lower())                          

undescribed_compounds_set = set()
duplicated_compounds = []
for compound in undescribed_compounds:
    compound = fix_encoding(compound)
    compound = compound.strip()
    undescribed_compounds_set.add(compound)

print('described compounds: ', len(described_compounds.keys()))
print('undescribed_compounds: ', len(undescribed_compounds))
print('unknown compounds: ', len(unknown_compounds))
print('repeated compounds: ', len(repeated_compounds))
print('total compounds: ', len(set(compounds_list)))

# view any missing compounds
modeled_compounds = undescribed_compounds_set.union(set(described_compounds.keys())) 
modeled_compounds_set = set()
for compound in modeled_compounds:
    compound = fix_encoding(compound)
    compound = compound.strip()
    modeled_compounds_set.add(compound)
    
print('\n')
print('modeled compounds: ', len(modeled_compounds_set))
missing_compounds = set(compounds_list) - modeled_compounds_set
print('missing compounds: ', len(set(missing_compounds)))
for compound in missing_compounds:
    print(compound)
print('\n\n')
    
# export the acquired sets of described and undescribed compounds
'''with open('{}_APF_described compounds in ModelSEED_01.json'.format(date.today()), 'w') as output:
    json.dump(described_compounds, output, indent = 5)'''

'''with open('{}_APF_undescribed compounds in ModelSEED_01.csv'.format(date.today()), 'w', encoding="utf-8") as output:
    undescribed_dataframe = pandas.DataFrame(undescribed_compounds, columns = ['undescribed compounds'])
    undescribed_dataframe.fillna('', inplace = True)
    undescribed_dataframe.to_csv(output)'''

'''with io.open('2021-05-09_APF_NIST compounds absent from ModelSEED_01.csv', 'w', encoding="utf-8") as output:
    undescribed_compund_csv = pandas.DataFrame(undescribed_compounds)
    undescribed_compund_csv.to_csv(output)'''

# define functions that parse the data later
def isfloat(entry):
    try:
        value = float(entry)
        return True
    except ValueError:
        value = ''
        return False
    
def average(list_entry):
    if len(list_entry) > 0:
        average =  sum(list_entry) / len(list_entry)
    else:
        average = 'nan'
    return average

# define the complete set of enzymes that are described by the database
enzymes = set()
for enzyme, information in nist.items():
    enzymes.add(enzyme.lower())
    
# arrange the data for the described compounds
described_enzyme_data = {}
for enzyme, information in nist.items():
    if enzyme.lower() in enzymes:
        
        # determine the set of unique reactions for each enzyme
        unique_reactions = set()
        for reaction in information['reaction']:
            unique_reactions.add(reaction)
        
        # calculate the average temperature 
        temperatures = []
        for temperature in information['experimental temperatures']:
            temperature = re.sub('(~)', '', temperature)
            if isfloat(temperature):
                temperatures.append(float(temperature))
        average_temperature = average(temperatures)
            
        # calculate the average ph 
        phs = []
        for ph in information['experimental phs']:
            ph = re.sub('(~|nan)', '', ph)
            if isfloat(ph):
                phs.append(float(ph))
        average_ph = average(phs)
                
    described_enzyme_data[enzyme] = {'unique reactions': unique_reactions,
                                    'average keq': information['Keq']['keq average'],
                                    'average enthalpy': information['Enthalpy']['enthalpy average'],
                                    'average temperature kelvin': average_temperature,
                                    'average pH': average_ph}

    
# export the dictionary of unique reactions and data for each enzyme
'''with open('{}_APF_ModelSEED described enzymes_01.json'.format(date.today()), 'w') as output:
    json.dump(described_enzyme_data, output, indent = 4)'''


# the reaction strings are completely substituted with ModelSEED compound IDs
described_compounds_list = []
for compound in described_compounds:
    described_compounds_list.extend([compound.lower()] + list(described_compounds[compound]['alternative names']))
described_compounds_string = ',\t'.join(list(set(described_compounds_list))) 
'''with open('described compounds matrix.txt', 'w') as output:
    output.write('\n'.join(described_compounds_list))
    output.close()'''
    
total_complete_reactions = []
exchanged_reactions = set()
to_modelseed_reactions = {}
reactions = {}

reaction_id_pairs = {}
id_number = 0
repeated_reactions = set()
for enzyme, information in described_enzyme_data.items():
    complete_reactions = []
    described_reactions = set()
    undescribed_reactions = set()
    print('\n', enzyme)
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)

        # parse the reactants and products for the specified reaction string
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            stoich_coefficient = re.compile('(\d\s|\d\/\d\s)')
            for element in reactants_list:
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = stoich_coefficient.findall(element)[0]
                    if re.search('\/', coefficient):
                        numerator = re.findall('\d', coefficient)[0]
                        denominator = re.findall('\d', coefficient)[1]
                        numerical_coefficient = float(numerator) / float(denominator)
                        coefficient = '({}) '.format(numerical_coefficient)
                    else:
                        coefficient = '({}) '.format(re.findall('\d', coefficient)[0])
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in described_compounds_list:
                    for compound, aliases in described_compounds.items():
                        if element.lower() == compound.lower() or element in aliases['alternative names']:
                            reactants.append(coefficient + aliases['ModelSEED ID'])
                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = stoich_coefficient.findall(element)[0]
                    if re.search('\/', coefficient):
                        numerator = re.findall('\d', coefficient)[0]
                        denominator = re.findall('\d', coefficient)[1]
                        numerical_coefficient = float(numerator) / float(denominator)
                        coefficient = '({}) '.format(numerical_coefficient)
                    else:
                        coefficient = '({}) '.format(re.findall('\d', coefficient)[0])
                else:
                    coefficient = ''
                    
                element = re.sub(stoich_coefficient, '', element)
                if element in described_compounds_list:
                    for compound, aliases in described_compounds.items():
                        if element.lower() == compound.lower() or element in aliases['alternative names']:
                            products.append(coefficient + aliases['ModelSEED ID'])
                else:
                    products.append(element)

            compounds = reactants + products
            compounds_list = reactants_list + products_list
        
        else:
            print('incorrect reaction: ', reaction)
            compounds = None
            
        # assemble the reaction string
        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
        elif len(reactants) == 1:
            reactants_string = reactants[0]
        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        if len(products) > 1:
            products_string = ' + '.join(products)
        elif len(products) == 1:
            products_string = products[0]
        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        reaction_string = reactants_string + ' <=> ' + products_string
            
        # format the reaction string according to ModelSEED conventions of the stoichiometric coefficients and the compartment signifier                        
        reaction_string = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'({}) '.format(coefficient), reaction_string)
        reaction_string = re.sub('(?<=\d)(\s)(?=\))', '', reaction_string)
            
        if re.search('(?<=>|\+)(\s)(?=\w)', reaction_string):
            reaction_string = re.sub('(?<=>|\+)(\s)(?=\w)', ' (1) ', reaction_string)

        if not re.search('^( \d|\()', reaction_string):
            reaction_string = '(1) ' + reaction_string
            
        reaction_string = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction_string)
        reaction_string += '[0]'

        exchanged_reactions.add(reaction_string)
        if compounds is not None:
           
            # add the ModelSEED ID-exchange reaction string
            if all('cpd' in compound for compound in compounds):
                if reaction_string not in complete_reactions:
                    print(reaction_string)
                    complete_reactions.append(reaction_string)
                    
            # determine the set of described and undescribed compounds for each enzyme
            clean_compounds_list = []
            for compound in compounds_list:
                compound = compound.strip()
                compound = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', compound)
                compound = re.sub('(\d\s|\d\/\d\s)', '', compound)
                if compound not in compounds_list:
                    clean_compounds_list.append(compound)
            
            described = all(compound in described_compounds_list for compound in clean_compounds_list)
            if described and reaction not in (undescribed_reactions and described_reactions):
                described_reactions.add(reaction)
            elif not described and reaction not in (undescribed_reactions and described_reactions):
                undescribed_reactions.add(reaction)
            else:
                repeated_reactions.add(reaction)

    # create unique reaction IDs for each compound
    reaction_ids = []
    for reaction in complete_reactions:
        if reaction not in reaction_id_pairs.values():
            reaction_id = 'TECRDB{}'.format(id_number)
            reaction_id_pairs[reaction_id] = reaction
            id_number += 1
        else:
            index = list(reaction_id_pairs.values()).index(reaction)
            reaction_id = list(reaction_id_pairs.keys())[index]
            print('--> repeated reaction, {}.'.format(reaction))
            
        reaction_ids.append(reaction_id)

    # add the enzyme-reactions associations to the respective dictionaries
    total_complete_reactions += complete_reactions    
    reactions[enzyme] = {'completely described reactions': {'reactions':complete_reactions, 'ids':reaction_ids},
                         'undescribed reactions': list(undescribed_reactions)}
    
# the parsed data is viewed
print('\n')
print('Total enzymes: {}'.format(len(described_enzyme_data.keys())))
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Total complete reactions: {}'.format(len(total_complete_reactions)))  
print('Total complete reactions set: {}'.format(len(set(total_complete_reactions))))
print('Previously complete reactions: {}'.format(len(set(list(tecrdb_reactions.keys())))))

print('\n')

completely_described = 0
undescribed = 0
for enzyme, data in reactions.items():
    if len(data['completely described reactions']) != 0 and len(data['undescribed reactions']) == 0:
        completely_described += 1
    else:
        undescribed += 1
        
print('enzymes with completely described reactions: ', completely_described)
print('enzymes with >0 undescribed reactions: ', undescribed)
print('repeated reactions for an enzyme: ', len(repeated_reactions))

# the parsed data is exported
with open('{}_TECR_reactions_in_ModelSEED.json'.format(date.today()), 'w') as output:
    json.dump(reactions, output, indent = 4)


# export the ModelSEED formatted reactions into a TSV file 
print('\n')
tsv_lines = []
for reaction_id in reaction_id_pairs:
    tsv_lines.append('\t'.join([reaction_id, reaction_id_pairs[reaction_id]]))
print('Total reactions in TSV: {}'.format(len(tsv_lines)))  
        
'''tsv_file = open('{}_TECR_to_ModelSEED_ID-rxn.tsv'.format(date.today()), 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()'''

identified alias:  l-aspartate
identified abbreviation:  fumarate
identified abbreviation:  ammonia
identified alias:  h2o
--> undescribed compound in ModelSEED, (s)-malate 
identified alias:  2-phospho-d-glycerate
identified alias:  phosphoenolpyruvate
identified abbreviation:  d-fructose 1,6-bisphosphate
identified alias:  glycerone phosphate
identified alias:  3-phospho-d-glycerate
identified alias:  phospho-l-arginine
identified alias:  l-arginine
identified alias:  orthophosphate
identified alias:  phosphocreatine
identified abbreviation:  creatine
--> undescribed compound in ModelSEED, (r)-3-phosphoglycerate 
--> undescribed compound in ModelSEED, (r)-glycerate 
identified alias:  pyruvate
identified abbreviation:  d-fructose 1-phosphate
identified alias:  d-fructose 6-phosphate
identified alias:  d-fructose 6-phosphate
identified abbreviation:  d-fructose
identified alias:  d-galactose 6-phosphate
identified abbreviation:  d-galactose
identified abbreviation:  d-glucose 6-phosph

identified alias:  d-glucosamine 6-phosphate
identified abbreviation:  d-glucosamine 1-phosphate
--> undescribed compound in ModelSEED, α-d-glucose 1,6-diphosphate 
--> undescribed compound in ModelSEED, 6-thioglucose 6-phosphate 
--> undescribed compound in ModelSEED, 6-thioglucose 1-phosphate 
identified alias:  sucrose
identified alias:  dadp
identified alias:  damp
identified alias:  datp
--> undescribed compound in ModelSEED, adp-n1-oxide 
--> undescribed compound in ModelSEED, atp-n1-oxide 
identified abbreviation:  adenosine 5'-tetraphosphate
identified alias:  citrate
identified alias:  cis-aconitate
identified alias:  isocitrate
identified alias:  isocitrate
identified abbreviation:  acetate
identified abbreviation:  acetyl phosphate
identified alias:  3-phospho-d-glyceroyl phosphate
identified alias:  (aminomethyl)phosphonate
--> undescribed compound in ModelSEED, (3-phospho-d-glycerol-aminomethyl)phosphonate 
--> undescribed compound in ModelSEED, (chloroethyl)phosphonate 
-

identified alias:  adpglucose
identified alias:  quinate
identified alias:  5-dehydroquinate
identified alias:  shikimate
identified abbreviation:  5-dehydroshikimate
identified alias:  d-glucono-1,5-lactone 6-phosphate
identified alias:  sedoheptulose 7-phosphate
identified abbreviation:  d-erythrose 4-phosphate
identified alias:  d-glyceraldehyde-3-phosphate
--> undescribed compound in ModelSEED, (r)-lactate 
identified abbreviation:  itp
identified abbreviation:  idp
--> undescribed compound in ModelSEED, atp4- 
identified abbreviation:  utp
identified alias:  ump
identified abbreviation:  orotidine 5'-phosphate
identified abbreviation:  orotate
--> undescribed compound in ModelSEED, 5-phospho-α-d-ribose 1-diphosphate 
--> undescribed compound in ModelSEED, thiopyrophosphate 
--> undescribed compound in ModelSEED, phosphoribosyl-1-o-(2-thiodiphosphate) 
--> undescribed compound in ModelSEED, (l-lysine)n 
--> undescribed compound in ModelSEED, (n-1) h2o 
identified alias:  l-lysine
-

--> undescribed compound in ModelSEED, d-erythrose 4-phospahte 
identified abbreviation:  2-hydroxy-3-oxopropanoate
identified alias:  dttp
--> undescribed compound in ModelSEED, dtdpglucose 
identified alias:  2-methyl-4-amino-5-hydroxymethylpyrimidine diphosphate
--> undescribed compound in ModelSEED, 4-methyl-5-(2-phosphonoxyethyl)-thiazole 
identified alias:  thiamine monophosphate
--> undescribed compound in ModelSEED, nω-phosphotaurocyamine 
--> undescribed compound in ModelSEED, l-tyrosylglycinamide 
identified alias:  l-tyrosine
identified alias:  benzoic acid
identified alias:  4-methyl-2-oxopentanoate
identified alias:  l-o-phosphoserine
identified abbreviation:  propanoyl-coa
identified alias:  methylmalonyl-coa
identified alias:  methylmalonyl-coa
identified abbreviation:  d-ribose 1-phosphate
identified alias:  6-phospho-2-dehydro-3-deoxy-d-gluconate
identified alias:  2-dehydro-3-deoxy-d-galactonate 6-phosphate
identified alias:  d-galactono-1,4-lactone
identified abbrevi

--> undescribed compound in ModelSEED, adenosine 3':5'-(cyclic)phosphate 
--> undescribed compound in ModelSEED, guanosine 3':5'-(cyclic)phosphate 
--> undescribed compound in ModelSEED, 2'-deoxyadenosine 3':5'-(cyclic)phosphate 
identified abbreviation:  2'-deoxyadenosine 5'-monophosphate
--> undescribed compound in ModelSEED, inosine 3':5'-(cyclic)phosphate 
--> undescribed compound in ModelSEED, uridine 3':5'-(cyclic)phosphate 
identified abbreviation:  \(\+\)-camphor
identified abbreviation:  \(\+\)-5-exo-hydroxycamphor
--> undescribed compound in ModelSEED, (r)-5-phosphomevalonate 
--> undescribed compound in ModelSEED, (r)-5-diphosphomevalonate 
--> undescribed compound in ModelSEED, atp-α-s 
--> undescribed compound in ModelSEED, adp-α-s 
--> undescribed compound in ModelSEED, atp-γ-s 
--> undescribed compound in ModelSEED, 2-thio-5-diphosphomevalonate 
identified abbreviation:  l,l-2,6-diaminoheptanedioate
identified alias:  meso-diaminoheptanedioate
--> undescribed compound in

--> undescribed compound in ModelSEED, nω-phospholombricine 
identified abbreviation:  disaccharide
identified abbreviation:  tetrasaccharide
--> undescribed compound in ModelSEED, trisaccharide 
identified alias:  d-arabitol
identified alias:  palatinose
identified alias:  d-turanose
identified alias:  chorismate
identified abbreviation:  isochorismate
identified abbreviation:  2-amino-2-deoxyisochorismate
identified alias:  retinyl palmitate
identified abbreviation:  retinol
identified alias:  palmitate
identified abbreviation:  4-amino-4-deoxychorismate
--> undescribed compound in ModelSEED, maltosyl-β-cyclomaltoheptaose 
identified alias:  diolein
identified alias:  oleic acid
--> undescribed compound in ModelSEED, monoolein 
identified alias:  triolein
--> undescribed compound in ModelSEED, n-decanoic acid glycerol diester 
--> undescribed compound in ModelSEED, n-decanoic acid 
--> undescribed compound in ModelSEED, n-decanoic acid glycerol monoester 
--> undescribed compound in 

(1) cpd00023[0] + (1) cpd00006[0] + (1) cpd00001[0] <=> (1) cpd00024[0] + (1) cpd00005[0] + (1) cpd00013[0]
(1) cpd00023[0] + (1) cpd00003[0] + (1) cpd00001[0] <=> (1) cpd00024[0] + (1) cpd00004[0] + (1) cpd00013[0]
--> repeated reaction, (1) cpd00023[0] + (1) cpd00003[0] + (1) cpd00001[0] <=> (1) cpd00024[0] + (1) cpd00004[0] + (1) cpd00013[0].

 arginosuccinate lyase

 dipeptidyl-peptidase I

 3(or 17)_-hydroxysteroid dehydrogenase

 ribose-5-phosphate isomerase
(1) cpd00101[0] <=> (1) cpd00171[0]

 nucleoside-diphosphate kinase
(1) cpd00002[0] + (1) cpd00090[0] <=> (1) cpd00008[0] + (1) cpd00068[0]
(1) cpd00002[0] + (1) cpd00031[0] <=> (1) cpd00008[0] + (1) cpd00038[0]

 ketotetraose-phosphate aldolase
(1) cpd02148[0] <=> (1) cpd00055[0] + (1) cpd00095[0]

 butyryl-CoA dehydrogenase
(1) cpd00120[0] + (1) cpd00120[0] + (1) cpd01206[0] <=> (1) cpd00650[0] + (1) cpd00650[0] + (1) cpd23609[0]

 glutamate-ammonia ligase

 dihydroorotase
(1) cpd00282[0] + (1) cpd00001[0] <=> (1) cpd00343[

 methylaspartate mutase
(1) cpd02273[0] <=> (1) cpd00023[0]

 oxamate carbamoyltransferase
(1) cpd00146[0] + (1) cpd01027[0] <=> (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd22507[0]

 methylmalonyl-CoA mutase

 tetrahydroxypteridine cycloisomerase
(1) cpd02033[0] <=> (1) cpd02112[0]

 thioredoxin reductase (NADPH)

 carnitine O-palmitoyltransferase
(1) cpd00134[0] + (1) cpd00266[0] <=> (1) cpd22528[0] + (1) cpd01915[0]

 mannose-1-phosphate guanylyltransferase

 D-lyxose ketol-isomerase
(1) cpd00368[0] <=> (1) cpd00259[0]
--> repeated reaction, (1) cpd00368[0] <=> (1) cpd00259[0].

 oxaloacetate tautomerase
(1) cpd00032[0] + (1) cpd00032[0] <=> (1) cpd02469[0]

 pancreatic ribonuclease

 deoxy)nucleoside-phosphate kinase

 papain

 rhamnulose-1-phosphate aldolase
(1) cpd00831[0] <=> (1) cpd00095[0] + (1) cpd00334[0]

 2-deoxy-D-gluconate dehydrogenase
(1) cpd01795[0] + (1) cpd00003[0] <=> (1) cpd02441[0] + (1) cpd00004[0]

 ureidoglycolate lyase

 N-acylglucosamine 2-e

(1) cpd01756[0] + (1) cpd00001[0] <=> (1) cpd00060[0] + (1) cpd29968[0]
(1) cpd33748[0] + (1) cpd00001[0] <=> (1) cpd29968[0] + (1) cpd00035[0]

 sorbitol-6-phosphate dehydrogenase (NADP+)

 pyrroline-5-carboxylate reductase

 GDP-D-mannose 3,5-epimerase

 serine-pyruvate transaminase
(1) cpd00035[0] + (1) cpd00145[0] <=> (1) cpd00054[0] + (1) cpd00020[0]

 glycerol kinase
(1) cpd00002[0] + (1) cpd00100[0] <=> (1) cpd00008[0] + (1) cpd00080[0]

 hypoxanthine phosphoribosyltransferase
(1) cpd00126[0] + (1) cpd00226[0] <=> (1) cpd00114[0] + (1) cpd00207[0]

 lactoylglutathione lyase

 malate dehydrogenase (decarboxylating)

 formate dehydrogenase (NADP+)
(1) cpd00047[0] + (1) cpd00006[0] + (1) cpd00001[0] <=> (1) cpd29976[0] + (1) cpd00005[0]

 dimethylmaleate hydratase

 thiamine pyridinylase

 methylenetetrahydrofolate dehydrogenase (NAD+)
(1) cpd00125[0] + (1) cpd00003[0] <=> (1) cpd00347[0] + (1) cpd00004[0]

 3-hydroxybutyryl-CoA dehyratase

 Ca2+-transporting ATPase
(1) cpd00002[0]

"tsv_file = open('{}_TECR_to_ModelSEED_ID-rxn.tsv'.format(date.today()), 'w')\ntsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))\ntsv_file.close()"

# Master TECR file creation

In [ ]:
from datetime import date
from numpy import nan 
import pandas
import re

# import the spreadsheets
elad_file = pandas.read_csv('TECRDB_Elad_Noor.csv')
our_file = pandas.read_csv('2021-08-04_vetted & reorganized NIST database_01.csv')

# contrast enzyme names between our spreadsheets
our_enzyme_names = set(our_file['Enzyme:'])
striped_enzymes = set()
for enzyme in our_enzyme_names:
    enzyme = enzyme.strip()
    striped_enzymes.add(enzyme)
'''print('Our enzymes: ', len(striped_enzymes), '\n', striped_enzymes)'''

elad_enzyme_names = set(elad_file['enzyme_name'])  

extra_enzymes = striped_enzymes - elad_enzyme_names
missing_enzymes = elad_enzyme_names - striped_enzymes
print('\nExtra enzymes in our file: ', len(extra_enzymes), '\n', extra_enzymes)
print('\nMissing enzymes in our file: ', len(missing_enzymes), '\n', missing_enzymes)
same_enzymes = striped_enzymes.intersection(elad_enzyme_names)

# contrast references between our spreadsheets
our_references = set(our_file['Reference ID:'])
striped_references = set()
for reference in our_references:
    substituted_reference = re.sub('_.+', '', reference)
    striped_references.add(substituted_reference)

elad_references = set(elad_file['reference'])

extra_references = striped_references - elad_references
missing_references = elad_references - striped_references
print('\nExtra enzymes in our file: ', len(extra_references), '\n', extra_references)
print('\nMissing enzymes in our file: ', len(missing_references), '\n', missing_references)



# establish standard IDs for each TECR datum
column_nans = [nan for row in range(len(our_file))]
our_file.insert(0, 'Standard_ID', column_nans)

standard_ids = [id for id in range(len(elad_file))]
elad_file.insert(0, 'standard_id', standard_ids)
elad_file.to_csv('TECRDB_Elad_Noor.csv')


# parse Elad's spreadsheet with our spreadsheet
our_file.insert(2, 'KEGG Reaction:', column_nans)
display(our_file.head(5))

for elad_index, elad_row in elad_file.iterrows():
    elad_enzyme = elad_row['enzyme_name']
    if elad_enzyme in same_enzymes:
        pass                                          # logic must be constructed such that only the corresponding row is expanded, and not every row that matches enzyme name 
                                 
    # add undescribed enzymes and citations
    else:
        pass 
    
        # define the column values 
        add_enzyme = elad_row['enzyme_name']
        add_reference = elad_row['reference']
        add_method = elad_row['method']
        add_ec = elad_row['EC']
        add_kegg_reaction = elad_row['reaction']
        add_reaction = elad_row['description']
        add_keg_reaction = elad_row['reaction']
        add_pmg = elad_row['p_mg']
        add_k = elad_row['K_prime']
        if add_k is nan:
            add_k = elad_row['K']
        add_temperature = elad_row['temperature']
        add_ionic_strength = elad_row['ionic_strength']
        add_ph = elad_row['p_h']
        
        # add a new row at the end of our dataframe, according to our column organization
        our_file.loc[len(our_file.index)] = [add_enzyme, add_reaction, add_kegg_reaction, nan, add_reference, add_temperature, add_ph, add_k, nan, add_method, nan, add_pmg, add_ec, nan, nan, nan, add_ionic_strength, nan]
        
# export the combined master file
display(our_file.tail(5))
output = '{}_master_TECR_1.csv'.format(date.today())
our_file.to_csv(output)

In [60]:
from datetime import date
from numpy import nan 
import pandas
import re

# import the spreadsheets
elad_file = pandas.read_csv('TECRDB_Elad_Noor.csv')
master_file = pandas.read_csv('2021-08-08_master_TECR_1.csv')
for column in master_file:
    if re.search('Unnamed', column):
        del master_file[column]

# contrast references between our spreadsheets
our_references = set(master_file['Reference ID:'])
striped_references = set()
for reference in our_references:
    substituted_reference = re.sub('_.+', '', reference)
    striped_references.add(substituted_reference)

elad_references = set(elad_file['reference'])

extra_references = striped_references - elad_references
missing_references = elad_references - striped_references
print('\nExtra references in the master file: ', len(extra_references), '\n', extra_references)
print('\nMissing references in the master file: ', len(missing_references), '\n', missing_references)
same_references = striped_references.intersection(elad_references)

display(master_file.tail(5))

for elad_index, elad_row in elad_file.iterrows():
    elad_reference = elad_row['reference']
    if elad_reference in same_references:
        continue                                          # logic must be constructed such that only the corresponding row is expanded, and not every row that matches enzyme name 

                                 
    # add undescribed enzymes and citations
    else:
        # define the column values 
        add_id = elad_row['standard_id']
        add_enzyme = elad_row['enzyme_name']
        add_reference = elad_row['reference']
        add_method = elad_row['method']
        add_ec = elad_row['EC']
        add_kegg_reaction = elad_row['reaction']
        add_reaction = elad_row['description']
        add_keg_reaction = elad_row['reaction']
        add_pmg = elad_row['p_mg']
        add_k = elad_row['K_prime']
        if add_k is nan:
            add_k = elad_row['K']
        add_temperature = elad_row['temperature']
        add_ionic_strength = elad_row['ionic_strength']
        add_ph = elad_row['p_h']
        
        # add a new row at the end of our dataframe, according to our column organization
        master_file.loc[len(master_file.index)] = [add_id, add_enzyme, add_reaction, add_kegg_reaction, nan, add_reference, add_temperature, add_ph, add_k, nan, add_method, nan, add_pmg, add_ec, nan, nan, nan, add_ionic_strength, nan]
        
# export the combined master file
display(master_file.tail(5))
output = '2021-08-08_master_TECR_1.csv'
master_file.to_csv(output)


Extra references in the master file:  97 
 {'89IMA', '76TRI/PAR', '73STU/GER', '84REK/RUM', '95WIS/KUS', '97TEW/GOL', '84PUZ/GOR', '61GAW/GLA', '86MEY/BRO', '65CAR/KIR', '80JAF/COH', '68AYL/SNE2', '36MEY/LOH', '98LOV/LAU', '71MCC/CHA', '80ELM/HAS', '52DOB/STU', '65BOY/BAR', '54GRE/MII', '75KAP/BAR', '77ANN/WAL', '95CHE/ARM', '87KUC/MIZ', '98KIM/VOE', '70HER', '95LEE/WAL', '88BED/HAD', '57GRE/LIP', '66CAR/HUL', '91KNI/SEM', '89JOH/HED', '97SAL/GOD', '86PRU/TEN', '96TEW/SCH', '94CHI/KIR', '53MAH/WAK', '68MAY/AND', '63JEN/CAP', '62GOL/WAG', '63EHR/MAR', '69ROS/HAM', '55STU2', '95TEW/SCH', '66TOO/WAK', '75SHI/BEA', '93BLI/MAR', '98TEW', '96ARA/RUZ', '53STU', '91HOR/UEH', '76GRE/BRI', '96HIR/MAY', '56HAU', '98THO/JOR', '59VAG/EAR', '56TOM', '63OKA', '61RAW/WAD', '64BOJ/GAU', '92LOR/TRA', '51FRU/JOH', '99TEW/SCH', '63DAY/WIL', '84OYA/IRI', '77GRI/LOC', '79LAB/DEB', '36VEI', '56FOR/GUT', '52DOB/FRU', '80SLI/BOL', '73VID/UDE', '65BAH/CAT', '86NAK/KIM', '92IBO/OBO', ' ', '85HER/AIY', '92BLA/GU

,standard_id,Enzyme:,Reaction:,KEGG Reaction:,Reference:,Reference ID:,T [K],pH,Keq,Km,Method:,Buffer:,Experimental conditions,EC Value:,solutes [mol / kg],solutes [mol / dm^3],Ionic strength [mol / dm^3],Ionic strength [mol / kg],Enthalpy [kJ / mol]
4933,4504.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,298.15,6.7,6.98e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4934,4505.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,280.13,7.0,2.46e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4935,4506.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,298.15,6.4,0.000111,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4936,4507.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,307.93,7.0,2.74e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4937,4508.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,298.15,7.4,1.58e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN


,standard_id,Enzyme:,Reaction:,KEGG Reaction:,Reference:,Reference ID:,T [K],pH,Keq,Km,Method:,Buffer:,Experimental conditions,EC Value:,solutes [mol / kg],solutes [mol / dm^3],Ionic strength [mol / dm^3],Ionic strength [mol / kg],Enthalpy [kJ / mol]
4933,4504.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,298.15,6.7,6.98e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4934,4505.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,280.13,7.0,2.46e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4935,4506.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,298.15,6.4,0.000111,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4936,4507.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,307.93,7.0,2.74e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN
4937,4508.0,pyruvate kinase,ATP(aq) + pyruvate(aq) = phosphoenolpyruvate(a...,kegg:C00002 + kegg:C00022 = kegg:C00074 + kegg...,NaN,02DOB/HIT,298.15,7.4,1.58e-05,NaN,spectrophotometry and enzymatic assay,NaN,3.0,2.7.1.40,NaN,NaN,NaN,0.25,NaN


PermissionError: [Errno 13] Permission denied: '2021-08-08_master_TECR_1.csv'

In [14]:
from datetime import date
from numpy import nan 
import pandas
import re

empty = [nan, 'nan', None, ' ', 'NaN']

# import the spreadsheets
elad_file = pandas.read_csv('TECRDB_Elad_Noor.csv')
master_file = pandas.read_csv('2021-08-08_master_TECR_1.csv')
for column in master_file:
    if re.search('Unnamed', column):
        del master_file[column]
        
# create a matching and redefining function
def redefine_master(master_file, master_row, elad_row):
    # match KEGG reactions
    if master_row['KEGG Reaction:'] in empty:
        master_row['KEGG Reaction:'] = elad_row['reaction']

    # match magnesium concentrations
    if float(master_pmg) == float(elad_pmg):
        pass            
    elif master_pmg in empty:
        master_row['Experimental conditions'] = '{} = -log[Mg+2]'.format(str(elad_row['p_mg']))
    elif elad_pmg not in empty:
        master_row['Experimental conditions'] = "".join([str(master_pmg), ' or {} (-log[Mg+2])'.format(str(elad_pmg))]) 
        print(master_index, '\t', 'pmg', '\t', master_row['Experimental conditions'])

    # match methods
    if master_method == elad_method:
        pass            
    elif master_method in empty:
        master_row['Method:'] = elad_method
    elif elad_pmg not in empty:
        master_row['Method:'] = "".join([master_method, elad_method]) 
        print(master_index, '\t', 'method', '\t', master_row['Method:'])

    # match EC values
    if master_ec == elad_ec:
        pass            
    elif master_ec in empty:
        master_row['EC Value:'] = elad_ec
    elif elad_pmg not in empty:
        master_row['EC Value:'] = "".join([master_ec, elad_ec])
        print(master_index, '\t', 'EC', '\t', master_row['EC Value:'])

    # match ionic strength concentrations
    if master_ionic_strength == elad_ionic_strength:
        pass            
    elif master_ionic_strength in empty:
        master_row['Ionic strength [mol / dm^3]'] = elad_ec
    elif elad_pmg not in empty:
        master_row['Ionic strength [mol / dm^3]'] = "".join([str(master_ionic_strength), elad_ionic_strength])
        print(master_index, '\t', 'ionic strength', '\t', master_row['Ionic strength [mol / dm^3]'])

    # match magnesium concentrations
    if master_id == elad_id:
        pass            
    elif master_id in empty:
        master_row['standard_id'] = elad_id
    elif elad_pmg not in empty:
        master_row['standard_id'] = "".join([str(master_id), str(elad_id)])
        print(master_index, '\t', 'standard ID', '\t', master_row['standard_id'])

    # print the datum pair for manual inspection
    announcement = '\nmatched pair:'
    print(announcement, '\n', '='*len(announcement))
    print('elad_index', elad_index)
    print('master_index', master_index, '\n')

    return True
    master_file.loc[len(master_file.index)] = [elad_id, elad_enzyme, elad_reaction, elad_kegg_reaction, nan, elad_reference, elad_temperature, elad_ph, elad_k, nan, elad_method, nan, elad_pmg, elad_ec, nan, nan, nan, elad_ionic_strength, nan]
        
        

unmatched_datums = []
unmatched_datums.append('Unmatched standard IDs')
display_count = 0
for elad_index, elad_row in elad_file.iterrows():
    matched_datum = False
    
    # define the Elad values for this datum 
    elad_id = elad_row['standard_id']
    elad_enzyme = elad_row['enzyme_name']
    elad_reference = elad_row['reference']
    elad_method = elad_row['method']
    elad_ec = elad_row['EC']
    elad_kegg_reaction = elad_row['reaction']
    elad_reaction = elad_row['description']
    elad_keg_reaction = elad_row['reaction']
    elad_pmg = elad_row['p_mg']
    elad_k = elad_row['K_prime']
    if elad_k is nan:
        elad_k = elad_row['K']
    elad_temperature = elad_row['temperature']
    elad_ionic_strength = elad_row['ionic_strength']
    elad_ph = elad_row['p_h']
    
    # determine the set of possible matches
    matching_master_subset = master_file.loc[(master_file['Enzyme:'] == elad_enzyme)]    #  & (master_file['Keq'] == elad_k) & (master_file['T [K]'] == elad_temperature) & (master_file['pH '] == elad_ph)
    if display_count < 5:
        display(matching_master_subset)
    display_count += 1
    
    for master_index, master_row in matching_master_subset.iterrows():   
        # define the master values for this datum 
        master_id = master_row['standard_id']
        master_reference_id = re.sub('_.+', '', master_row['Reference ID:'])
        master_method = master_row['Method:']
        master_ec = master_row['EC Value:']
        master_reaction = master_row['Reaction:']
        master_pmg = master_row['Experimental conditions']
        master_ionic_strength = master_row['Ionic strength [mol / dm^3]']
        master_temperature = master_row['T [K]']
        master_ph = master_row['pH ']
        master_k = master_row['Keq']

        #possible_match_count = 0
        if elad_reference == master_reference_id and elad_reaction == master_reaction and not matched_datum:
            if elad_temperature not in empty:
                if float(elad_temperature) != float(master_temperature):
                    print('temperature', '\t', elad_temperature, '\t', master_temperature)
                    continue
                    
            if elad_ph not in empty:
                if float(elad_ph) != float(master_ph):
                    print('ph', '\t', elad_ph, '\t', master_ph)
                    continue
                
            if elad_k not in empty:
                if float(elad_k) != float(master_k):
                    print('Keq', '\t', elad_k, '\t', master_k)
                    continue

            matched_datum = redefine_master(master_file, master_row, elad_row)
            break
            
    if not matched_datum:
        unmatched_datums.append(str(elad_index))
        
# export the unmatched datums
with open('2021-08-08_unmatched elad TECR datums.txt', 'w') as output:
    output.write('\n'.join(unmatched_datums))
    output.close()  
        
# export the combined master file
'''display(master_file.head(5))
csv_name = '2021-08-09_master_TECR_1.csv'
master_file.to_csv(csv_name)'''

,standard_id,Enzyme:,Reaction:,KEGG Reaction:,Reference:,Reference ID:,T [K],pH,Keq,Km,Method:,Buffer:,Experimental conditions,EC Value:,solutes [mol / kg],solutes [mol / dm^3],Ionic strength [mol / dm^3],Ionic strength [mol / kg],Enthalpy [kJ / mol]
4041,NaN,homoisocitrate dehydrogenase,"(1R,2S)-1-hydroxybutane-1,2,4-tricarboxylate(a...",kegg:C05662 + kegg:C00003 = kegg:C00322 + kegg...,NaN,07LIN/ALG,298.15,7.5,0.45,NaN,spectrophotometry,NaN,NaN,1.1.1.87,NaN,NaN,NaN,NaN,NaN


4041 	 standard ID 	 nan0

matched pair: 
elad_index 0
master_index 4041 



,standard_id,Enzyme:,Reaction:,KEGG Reaction:,Reference:,Reference ID:,T [K],pH,Keq,Km,Method:,Buffer:,Experimental conditions,EC Value:,solutes [mol / kg],solutes [mol / dm^3],Ionic strength [mol / dm^3],Ionic strength [mol / kg],Enthalpy [kJ / mol]


,standard_id,Enzyme:,Reaction:,KEGG Reaction:,Reference:,Reference ID:,T [K],pH,Keq,Km,Method:,Buffer:,Experimental conditions,EC Value:,solutes [mol / kg],solutes [mol / dm^3],Ionic strength [mol / dm^3],Ionic strength [mol / kg],Enthalpy [kJ / mol]
4695,3357.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,9.31,NaN,spectrophotometry,NaN,NaN,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4696,3358.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,310.15,7.5,10.8,NaN,spectrophotometry,NaN,NaN,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4697,3359.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,36.1,NaN,spectrophotometry,NaN,2.7,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4698,3360.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,46.1,NaN,spectrophotometry,NaN,2.52,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4699,3361.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,64.8,NaN,spectrophotometry,NaN,2.15,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4700,3362.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,48.3,NaN,spectrophotometry,NaN,2.7,4.2.1.3,NaN,NaN,NaN,0.044,NaN
4701,3363.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,77.7,NaN,spectrophotometry,NaN,2.1,4.2.1.3,NaN,NaN,NaN,0.044,NaN
4702,3364.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,37.9,NaN,spectrophotometry,NaN,2.7,4.2.1.3,NaN,NaN,NaN,0.07,NaN
4703,3365.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,67.3,NaN,spectrophotometry,NaN,2.1,4.2.1.3,NaN,NaN,NaN,0.07,NaN
4704,3366.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,21.5,NaN,spectrophotometry,NaN,3.0,4.2.1.3,NaN,NaN,NaN,0.15,NaN


,standard_id,Enzyme:,Reaction:,KEGG Reaction:,Reference:,Reference ID:,T [K],pH,Keq,Km,Method:,Buffer:,Experimental conditions,EC Value:,solutes [mol / kg],solutes [mol / dm^3],Ionic strength [mol / dm^3],Ionic strength [mol / kg],Enthalpy [kJ / mol]
4695,3357.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,9.31,NaN,spectrophotometry,NaN,NaN,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4696,3358.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,310.15,7.5,10.8,NaN,spectrophotometry,NaN,NaN,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4697,3359.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,36.1,NaN,spectrophotometry,NaN,2.7,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4698,3360.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,46.1,NaN,spectrophotometry,NaN,2.52,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4699,3361.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,64.8,NaN,spectrophotometry,NaN,2.15,4.2.1.3,NaN,NaN,NaN,0.1,NaN
4700,3362.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,48.3,NaN,spectrophotometry,NaN,2.7,4.2.1.3,NaN,NaN,NaN,0.044,NaN
4701,3363.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,77.7,NaN,spectrophotometry,NaN,2.1,4.2.1.3,NaN,NaN,NaN,0.044,NaN
4702,3364.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,37.9,NaN,spectrophotometry,NaN,2.7,4.2.1.3,NaN,NaN,NaN,0.07,NaN
4703,3365.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,67.3,NaN,spectrophotometry,NaN,2.1,4.2.1.3,NaN,NaN,NaN,0.07,NaN
4704,3366.0,aconitate hydratase,isocitrate(aq) = citrate(aq),kegg:C00311 = kegg:C00158,NaN,69BLA,298.15,7.5,21.5,NaN,spectrophotometry,NaN,3.0,4.2.1.3,NaN,NaN,NaN,0.15,NaN


...

4263 	 standard ID 	 nan3487

matched pair: 
elad_index 3487
master_index 4263 

4264 	 standard ID 	 nan3489

matched pair: 
elad_index 3489
master_index 4264 

Keq 	 53.0 	 40
4265 	 standard ID 	 nan3490

matched pair: 
elad_index 3490
master_index 4265 

4175 	 standard ID 	 nan3496

matched pair: 
elad_index 3496
master_index 4175 


matched pair: 
elad_index 3521
master_index 4804 

ph 	 8.0 	 7.3

matched pair: 
elad_index 3522
master_index 4805 

ph 	 8.0 	 7.3
Keq 	 300.0 	 1280.0

matched pair: 
elad_index 3523
master_index 4806 

ph 	 8.0 	 7.3
Keq 	 152.0 	 1280.0
Keq 	 152.0 	 300.0

matched pair: 
elad_index 3524
master_index 4807 

ph 	 8.0 	 7.3
Keq 	 650.0 	 1280.0
Keq 	 650.0 	 300.0
Keq 	 650.0 	 152.0

matched pair: 
elad_index 3525
master_index 4808 

ph 	 8.0 	 7.3
Keq 	 330.0 	 1280.0
Keq 	 330.0 	 300.0
Keq 	 330.0 	 152.0
Keq 	 330.0 	 650.0

matched pair: 
elad_index 3526
master_index 4809 

ph 	 8.0 	 7.3
Keq 	 158.0 	 1280.0
Keq 	 158.0 	 300.0
Keq 	 158.0 	

Keq 	 1.14 	 1.05
Keq 	 1.14 	 2
Keq 	 1.14 	 1.28
4275 	 standard ID 	 nan3620

matched pair: 
elad_index 3620
master_index 4275 

Keq 	 1.26 	 1.64
Keq 	 1.26 	 2.19
Keq 	 1.26 	 1.05
Keq 	 1.26 	 2
Keq 	 1.26 	 1.28
Keq 	 1.26 	 1.14
4276 	 standard ID 	 nan3621

matched pair: 
elad_index 3621
master_index 4276 

Keq 	 1.41 	 1.64
Keq 	 1.41 	 2.19
Keq 	 1.41 	 1.05
Keq 	 1.41 	 2
Keq 	 1.41 	 1.28
Keq 	 1.41 	 1.14
Keq 	 1.41 	 1.26
4277 	 standard ID 	 nan3622

matched pair: 
elad_index 3622
master_index 4277 

Keq 	 1.84 	 1.64
Keq 	 1.84 	 2.19
Keq 	 1.84 	 1.05
Keq 	 1.84 	 2
Keq 	 1.84 	 1.28
Keq 	 1.84 	 1.14
Keq 	 1.84 	 1.26
Keq 	 1.84 	 1.41
4278 	 standard ID 	 nan3623

matched pair: 
elad_index 3623
master_index 4278 

Keq 	 2.14 	 1.64
Keq 	 2.14 	 2.19
Keq 	 2.14 	 1.05
Keq 	 2.14 	 2
Keq 	 2.14 	 1.28
Keq 	 2.14 	 1.14
Keq 	 2.14 	 1.26
Keq 	 2.14 	 1.41
Keq 	 2.14 	 1.84
4279 	 standard ID 	 nan3624

matched pair: 
elad_index 3624
master_index 4279 

Keq 	 1.43 	 1.6

matched pair: 
elad_index 3909
master_index 4853 

temperature 	 333.15 	 298.15

matched pair: 
elad_index 3910
master_index 4854 

temperature 	 343.15 	 298.15
temperature 	 343.15 	 333.15

matched pair: 
elad_index 3911
master_index 4855 

temperature 	 323.15 	 298.15
temperature 	 323.15 	 333.15
temperature 	 323.15 	 343.15

matched pair: 
elad_index 3912
master_index 4856 

temperature 	 313.15 	 298.15
temperature 	 313.15 	 333.15
temperature 	 313.15 	 343.15
temperature 	 313.15 	 323.15

matched pair: 
elad_index 3913
master_index 4857 


matched pair: 
elad_index 3914
master_index 4858 


matched pair: 
elad_index 3915
master_index 4859 

temperature 	 333.15 	 343.15

matched pair: 
elad_index 3916
master_index 4860 

temperature 	 313.15 	 343.15
temperature 	 313.15 	 333.15

matched pair: 
elad_index 3917
master_index 4861 

temperature 	 323.15 	 343.15
temperature 	 323.15 	 333.15
temperature 	 323.15 	 313.15

matched pair: 
elad_index 3918
master_index 4862 

t


matched pair: 
elad_index 4049
master_index 4906 

Keq 	 nan 	 nan
4294 	 standard ID 	 nan4090

matched pair: 
elad_index 4090
master_index 4294 

Keq 	 2.46 	 2.91
4295 	 standard ID 	 nan4091

matched pair: 
elad_index 4091
master_index 4295 

Keq 	 1.07 	 2.91
Keq 	 1.07 	 2.46
4296 	 standard ID 	 nan4092

matched pair: 
elad_index 4092
master_index 4296 

Keq 	 0.91 	 2.91
Keq 	 0.91 	 2.46
Keq 	 0.91 	 1.07
4297 	 standard ID 	 nan4093

matched pair: 
elad_index 4093
master_index 4297 

Keq 	 0.98 	 2.91
Keq 	 0.98 	 2.46
Keq 	 0.98 	 1.07
Keq 	 0.98 	 0.91
4298 	 standard ID 	 nan4094

matched pair: 
elad_index 4094
master_index 4298 

Keq 	 2.69 	 2.91
Keq 	 2.69 	 2.46
Keq 	 2.69 	 1.07
Keq 	 2.69 	 0.91
Keq 	 2.69 	 0.98
4299 	 standard ID 	 nan4095

matched pair: 
elad_index 4095
master_index 4299 

Keq 	 1.21 	 2.91
Keq 	 1.21 	 2.46
Keq 	 1.21 	 1.07
Keq 	 1.21 	 0.91
Keq 	 1.21 	 0.98
Keq 	 1.21 	 2.69
4300 	 standard ID 	 nan4096

matched pair: 
elad_index 4096
master_

"display(master_file.head(5))\ncsv_name = '2021-08-09_master_TECR_1.csv'\nmaster_file.to_csv(csv_name)"

In [67]:
test = master_file.loc[master_file['Buffer:'] == 'phosphate']
print(test)
for index, row in test.iterrows():
    print(index)

      standard_id                                   Enzyme:  \
438           NaN                        fumarate hydratase   
439           NaN                        fumarate hydratase   
440           NaN                        fumarate hydratase   
441           NaN                        fumarate hydratase   
442           NaN                        fumarate hydratase   
678           NaN                             myosin ATPase   
744           NaN                                   trypsin   
747           NaN                                   trypsin   
750           NaN                                   trypsin   
759           NaN                                   trypsin   
1094          NaN                     glucuronate isomerase   
1116          NaN   methenyltetrahydrofolate cyclohydrolase   
1215          NaN           3-hydroxybutyrate dehydrogenase   
1720          NaN                      malate dehydrogenase   
1977          NaN               ribonuclease (unclassif

# Brainstorming

Conceptualizing a scrapping code for KEGG, which would unambiguously identify the NIST compounds.

In [49]:
import pandas

# import Elad's spreadsheet
elad_file = pandas.read_csv('TECRDB_Elad_Noor.csv')
display(elad_file.drop(elad_file.columns[[0, 1, 2]], axis = 1))

length = [datum for datum in range(len(elad_file))]
print(len(length))

,eval,EC,enzyme_name,reaction,description,K,K_prime,temperature,ionic_strength,p_h,p_mg
0,A,1.1.1.87,homoisocitrate dehydrogenase,kegg:C05662 + kegg:C00003 = kegg:C00322 + kegg...,"(1R,2S)-1-hydroxybutane-1,2,4-tricarboxylate(a...",NaN,0.450,298.15,NaN,7.5,NaN
1,C,3.5.4.9,methenyltetrahydrofolate cyclohydrolase,kegg:C00445 + kegg:C00001 = kegg:C00234,"5,10-methenyltetrahydrofolate(aq) + H2O(l) = 1...",NaN,4.200,298.15,NaN,6.5,NaN
2,B,4.2.1.3,aconitate hydratase,kegg:C00311 = kegg:C00158,isocitrate(aq) = citrate(aq),NaN,18.000,310.15,NaN,7.3,2.96
3,B,4.2.1.3,aconitate hydratase,kegg:C00311 = kegg:C00158,isocitrate(aq) = citrate(aq),NaN,25.000,310.15,NaN,7.3,2.80
4,B,4.2.1.3,aconitate hydratase,kegg:C00311 = kegg:C00158,isocitrate(aq) = citrate(aq),NaN,33.000,310.15,NaN,7.3,2.30
...,...,...,...,...,...,...,...,...,...,...,...
4539,E,4.1.1.21,phosphoribosylaminoimidazole carboxylase,kegg:C03373 + kegg:C00288 = kegg:C04751 + kegg...,Aminoimidazole ribotide + CO2 = 1-(5-Phospho-D...,NaN,1.800,310.15,NaN,7.8,NaN
4540,E,2.1.2.3,phosphoribosylaminoimidazolecarboxamide formyl...,kegg:C00234 + kegg:C04677 = kegg:C00101 + kegg...,10-Formyltetrahydrofolate + 1-(5'-Phosphoribos...,NaN,0.024,298.15,NaN,7.5,NaN
4541,E,4.3.-.-,formaldehyde condensation with THF,kegg:C00101 + kegg:C00067 = kegg:C00143,"THF(aq) + formaldehyde(aq) = 5,10-CH2-THF(aq)",NaN,7700.000,293.15,NaN,7.2,NaN
4542,E,4.1.2.43,3-Hexulose-6-Phosphate synthase,kegg:C00199 + kegg:C00067 = kegg:C06019,D-Ribulose 5-phosphate + Formaldehyde = D-arab...,NaN,25000.000,303.15,NaN,7.0,NaN


4544


In [ ]:
import BeautifulSoup
import selenium
import json

compounds = json.load(open('2021-04-29_APF_complete thermodynamically described compounds for KEGG_01.json'))

num = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

general_delay = 2  #seconds



def click_element_type(n_type):
    global driver
    element = driver.find_element_by_xpath(n_type)
    element.click()
    time.sleep(general_delay)
    
    
def select_dropdown_id(n_id, n_choice):
    global driver
    element = Select(driver.find_element_by_id(n_id))
    element.select_by_visible_text(n_choice)
    time.sleep(general_delay)

    
def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
    
def scrape(compound):
    global driver
    global enzymes
    
    driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
    driver.get("https://www.genome.jp/kegg/")
    
    time.sleep(general_delay)
    
    # search field
    text_area = driver.find_element_by_name("text")
    text_area.send_keys(compound)
    time.sleep(general_delay)
    click_element_id("//input[@value='Search']")
    time.sleep(general_delay)
    
    # display all compound button
    click_element_type("/html/body/form/a[@href='javascript:MoreSearch(\"compound\")']")
    
    # acquire the list of results
    '''possible_compounds = []
    for div in click_element_type("/html/body/form/div[@style='width_600px']"):'''
    url = driver.current_url
    
    compounds[compound] = "no"
    driver.close()
    return
    
    time.sleep(general_delay)
    
    select_dropdown_id("max", "100")
    element = Select(driver.find_element_by_id("max"))
    element.select_by_visible_text("100")
    
    time.sleep(general_delay)

    else:
        compounds[compound] = "no"
        driver.close()
        return
    
    enzymes[enzyme_name] = "yes"
    
    driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")
    
    time.sleep(15)
    
    click_element_id("excelExport")
    
    time.sleep(5)
    
    driver.close()
    

for compound in compounds:
    if compounds[compound] != "yes" and compounds[compound] != "no":
        flag = True
        while flag:
            try:
                scrape(compound)
                flag = False
            except:
                time.sleep(10)
            
        with open('compounds_to_collect_out.json', 'w') as outfile:
            json.dump(compounds, outfile, indent = 3)   
            outfile.close()

In [3]:
print('\xb0K')

°K


In [29]:
import re

compound = 'hi, yes, no'
re.sub('(^|, )(\S+)\w', '\u\1', ', {}, '.format(compound))
print(compound)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (<ipython-input-29-f8fcd23c4c81>, line 4)